In [2]:
import os
import time
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import from_json, col

# For use in Chapter 9 - Data Sources
# https://mvnrepository.com/artifact/org.xerial/sqlite-jdbc
packages = "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.4"

os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages {0} pyspark-shell".format(packages)
)

def logLevel(spark):
    # REF: https://stackoverflow.com/questions/25193488/how-to-turn-off-info-logging-in-spark
    sc = spark.sparkContext
    log4jLogger = sc._jvm.org.apache.log4j
    log4jLogger.Logger.getLogger("org").setLevel(log4jLogger.Level.ERROR)
    log = log4jLogger.LogManager.getLogger(__name__)
    log.warn("Custom Warning")


spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Demo") \
    .getOrCreate()


logLevel(spark)

spark.sparkContext.setLogLevel("ERROR")

In [3]:
print(spark.range(5000).where("id > 500").selectExpr("sum(id)").collect())

df = spark.read.format("csv") \
    .option("header", "true") \
    .option("mode", "FAILFAST") \
    .load("file:" + os.getenv("OPTION3_HOME") + "/data/training_set.csv")

df.show(5)

[Row(sum(id)=12372250)]
+---------+----------+--------+-----------+---------+--------+
|object_id|       mjd|passband|       flux| flux_err|detected|
+---------+----------+--------+-----------+---------+--------+
|      615|59750.4229|       2|-544.810303| 3.622952|       1|
|      615|59750.4306|       1|-816.434326| 5.553370|       1|
|      615|59750.4383|       3|-471.385529| 3.801213|       1|
|      615|59750.4450|       4|-388.984985|11.395031|       1|
|      615|59752.4070|       2|-681.858887| 4.041204|       1|
+---------+----------+--------+-----------+---------+--------+
only showing top 5 rows



# Read Stream

In [ ]:
# df = spark \
#     .readStream \
#     .format("kafka") \
#     .option("kafka.bootstrap.servers", "localhost:9092") \
#     .option("startingOffsets", "latest") \
#     .option("subscribe", "twitter_status_connect") \
#     .load()


# df.printSchema()

# topicSchema = StructType() \
#                 .add("schema", StringType()) \
#                 .add("payload", StringType())


# tweets = df.select(col("key").cast("string"),
#             from_json(col("value").cast("string"), topicSchema))

# print(type(tweets))

In [ ]:
# streamQuery = tweets.writeStream\
#                     .format("memory")\
#                     .queryName("tweets_data")\
#                     .outputMode("append")\
#                     .start()

In [ ]:
# print(streamQuery.isActive)

In [ ]:
# for seconds in range(10):
#     print("Refreshing....")
#     spark.sql("""
#       SELECT *
#       FROM tweets_data
#       """)\
#       .show(5)
#     time.sleep(2)

# print(type(spark.sql(""" SELECT * FROM tweets_data """)))

In [ ]:
# df = spark.sql(""" SELECT * FROM tweets_data """)

In [ ]:
# df.columns

In [ ]:
# df.select("jsontostructs(CAST(value AS STRING))").show(truncate=False)

In [ ]:
# streamQuery.stop()
# streamQuery.awaitTermination()

In [ ]:
# payload = df.toPandas()["jsontostructs(CAST(value AS STRING))"][0].asDict()['payload']

In [ ]:
# type(payload)

In [ ]:
# import json    # or `import simplejson as json` if on Python < 2.6
#
# json_string = payload
# obj = json.loads(json_string)

In [ ]:
# pp = json.loads(df.toPandas()["jsontostructs(CAST(value AS STRING))"][2].asDict()['payload'])
# pp

# Read

In [4]:
df = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "important_tweets") \
    .option("endingOffsets", """{"important_tweets":{"0":3}}""") \
    .load() \
    .selectExpr("CAST(value AS STRING) as tweets")

df.printSchema()

root
 |-- tweets: string (nullable = true)



In [5]:
df.show()

+--------------------+
|              tweets|
+--------------------+
|{"created_at":"Tu...|
|{"created_at":"Tu...|
|{"created_at":"Tu...|
+--------------------+



In [6]:
df.toPandas()['tweets'][0]

'{"created_at":"Tue May 26 16:15:01 +0000 2020","id":1265315511988084736,"id_str":"1265315511988084736","text":"On Channel 9, Scaling #dotnet for Apache Spark processing jobs with #Azure Synapse  On .NET from Cecil Phillip Rich\\u2026 https:\\/\\/t.co\\/Q3FnLXsn9H","source":"\\u003ca href=\\"https:\\/\\/buffer.com\\" rel=\\"nofollow\\"\\u003eBuffer\\u003c\\/a\\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2849837998,"id_str":"2849837998","name":"Azure Weekly","screen_name":"AzureWeekly","location":null,"url":"http:\\/\\/azureweekly.info","description":"Your free weekly @azure news fix. Subscribe at http:\\/\\/azureweekly.info Published every Sunday. Powered by @endjin","translator_type":"none","protected":false,"verified":false,"followers_count":14108,"friends_count":5404,"listed_count":315,"favourites_count":59,"statuses_count":23258,"created_at

In [7]:
import json

In [8]:
type(json.loads(df.toPandas()['tweets'][0])['user']['followers_count'])
json.loads(df.toPandas()['tweets'][0])['user']['followers_count']

14108

In [9]:
jsonTweet = json.loads(df.toPandas()['tweets'][0])

In [10]:
# if jsonTweet['coordinates'] is None:

In [12]:
df.schema

StructType(List(StructField(tweets,StringType,true)))

In [13]:
# df.select(col("tweets")).take(1)

In [14]:
import pyspark.sql.functions as psf


def parseJSONCols(df, *cols, sanitize=True):
    """Auto infer the schema of a json column and parse into a struct.

    rdd-based schema inference works if you have well-formatted JSON,
    like ``{"key": "value", ...}``, but breaks if your 'JSON' is just a
    string (``"data"``) or is an array (``[1, 2, 3]``). In those cases you
    can fix everything by wrapping the data in another JSON object
    (``{"key": [1, 2, 3]}``). The ``sanitize`` option (default True)
    automatically performs the wrapping and unwrapping.

    The schema inference is based on this
    `SO Post <https://stackoverflow.com/a/45880574)/>`_.

    Parameters
    ----------
    df : pyspark dataframe
        Dataframe containing the JSON cols.
    *cols : string(s)
        Names of the columns containing JSON.
    sanitize : boolean
        Flag indicating whether you'd like to sanitize your records
        by wrapping and unwrapping them in another JSON object layer.

    Returns
    -------
    pyspark dataframe
        A dataframe with the decoded columns.
    """
    res = df
    for i in cols:

        # sanitize if requested.
        if sanitize:
            res = (
                res.withColumn(
                    i,
                    psf.concat(psf.lit('{"data": '), i, psf.lit('}'))
                )
            )
        # infer schema and apply it
        schema = spark.read.json(res.rdd.map(lambda x: x[i])).schema
        res = res.withColumn(i, psf.from_json(psf.col(i), schema))

        # unpack the wrapped object if needed
        if sanitize:
            res = res.withColumn(i, psf.col(i).data)
    return res

In [15]:
res = parseJSONCols(df, 'tweets', sanitize=True)
res.schema

StructType(List(StructField(tweets,StructType(List(StructField(contributors,StringType,true),StructField(coordinates,StringType,true),StructField(created_at,StringType,true),StructField(display_text_range,ArrayType(LongType,true),true),StructField(entities,StructType(List(StructField(hashtags,ArrayType(StructType(List(StructField(indices,ArrayType(LongType,true),true),StructField(text,StringType,true))),true),true),StructField(symbols,ArrayType(StringType,true),true),StructField(urls,ArrayType(StructType(List(StructField(display_url,StringType,true),StructField(expanded_url,StringType,true),StructField(indices,ArrayType(LongType,true),true),StructField(url,StringType,true))),true),true),StructField(user_mentions,ArrayType(StructType(List(StructField(id,LongType,true),StructField(id_str,StringType,true),StructField(indices,ArrayType(LongType,true),true),StructField(name,StringType,true),StructField(screen_name,StringType,true))),true),true))),true),StructField(extended_tweet,StructType(

In [16]:
res.printSchema()

root
 |-- tweets: struct (nullable = true)
 |    |-- contributors: string (nullable = true)
 |    |-- coordinates: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- display_text_range: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- entities: struct (nullable = true)
 |    |    |-- hashtags: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |    |-- text: string (nullable = true)
 |    |    |-- symbols: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- urls: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |    |-- indices: array 

In [17]:
df.show()

+--------------------+
|              tweets|
+--------------------+
|{"created_at":"Tu...|
|{"created_at":"Tu...|
|{"created_at":"Tu...|
+--------------------+



In [18]:
res.show()

+--------------------+
|              tweets|
+--------------------+
|[,, Tue May 26 16...|
|[,, Tue May 26 16...|
|[,, Tue May 26 16...|
+--------------------+



In [19]:
res.select("tweets.created_at", "tweets.coordinates").show()

+--------------------+-----------+
|          created_at|coordinates|
+--------------------+-----------+
|Tue May 26 16:15:...|       null|
|Tue May 26 16:27:...|       null|
|Tue May 26 16:28:...|       null|
+--------------------+-----------+



In [20]:
res.select("tweets.created_at", "tweets.coordinates", "tweets.user.followers_count").show()

+--------------------+-----------+---------------+
|          created_at|coordinates|followers_count|
+--------------------+-----------+---------------+
|Tue May 26 16:15:...|       null|          14108|
|Tue May 26 16:27:...|       null|         133918|
|Tue May 26 16:28:...|       null|          74787|
+--------------------+-----------+---------------+



In [91]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Convenience function for turning JSON strings into DataFrames.
def jsonToDataFrame(json, schema=None):
  # SparkSessions are available with Spark 2.0+
  reader = spark.read
  if schema:
    reader.schema(schema)
  return reader.json(spark.sparkContext.parallelize([json]))

In [92]:
# Using a struct
schema = StructType().add("a", StructType().add("b", IntegerType()))
                          
events = jsonToDataFrame("""
{
  "a": {
     "b": 1
  }
}
""", schema)

In [93]:
events.select("a.b")

DataFrame[b: int]

In [94]:
events.show()

+---+
|  a|
+---+
|[1]|
+---+



In [97]:
df.select("tweets")

DataFrame[tweets: string]

In [81]:
from pyspark.sql.functions import explode
df.select(explode("tweets").alias("persons"))

AnalysisException: "cannot resolve 'explode(`tweets`)' due to data type mismatch: input to function explode should be array or map type, not string;;\n'Project [explode(tweets#69) AS persons#806]\n+- Project [cast(value#56 as string) AS tweets#69]\n   +- Relation[key#55,value#56,topic#57,partition#58,offset#59L,timestamp#60,timestampType#61] KafkaRelation(strategy=Subscribe[important_tweets], start=EarliestOffsetRangeLimit, end=SpecificOffsetRangeLimit(Map(important_tweets-0 -> 3)))\n"

In [77]:
from pyspark.sql.functions import from_json, col
# json_schema = spark.read.json(df.rdd.map(lambda row: row.json)).schema
json_schema = res.schema

In [78]:
df.withColumn('json', from_json(col('tweets'), json_schema)).show()

+--------------------+----+
|              tweets|json|
+--------------------+----+
|{"created_at":"Tu...|  []|
|{"created_at":"Tu...|  []|
|{"created_at":"Tu...|  []|
+--------------------+----+



In [84]:
df.select(col("tweets"), from_json(col('tweets'), json_schema)).show()

+--------------------+---------------------+
|              tweets|jsontostructs(tweets)|
+--------------------+---------------------+
|{"created_at":"Tu...|                   []|
|{"created_at":"Tu...|                   []|
|{"created_at":"Tu...|                   []|
+--------------------+---------------------+



In [87]:
df.printSchema()

root
 |-- tweets: string (nullable = true)



In [65]:
import pandas as pd

from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

# Declare the function and create the UDF
def multiply_func(tweets):
    #return tweets[0]['user']['followers_count']
    tweets.astype("string")
    return json.loads(tweets)['user']['followers_count']

In [66]:
multiply = pandas_udf(multiply_func, returnType=LongType())

In [67]:
df.select(col('tweets')).show()

+--------------------+
|              tweets|
+--------------------+
|{"created_at":"Tu...|
|{"created_at":"Tu...|
|{"created_at":"Tu...|
+--------------------+



In [69]:
type(col("tweets"))

pyspark.sql.column.Column

In [68]:
df.select(col('tweets'), multiply(col("tweets"))).show()

Py4JJavaError: An error occurred while calling o891.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 44.0 failed 1 times, most recent failure: Lost task 0.0 in stage 44.0 (TID 55, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.4.5/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/local/Cellar/apache-spark/2.4.5/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/Cellar/apache-spark/2.4.5/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 290, in dump_stream
    for series in iterator:
  File "<string>", line 1, in <lambda>
  File "/usr/local/Cellar/apache-spark/2.4.5/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 101, in <lambda>
    return lambda *a: (verify_result_lengt

In [ ]:
print(json.loads(df.toPandas()['tweets'][0]).get("followers_count"))

In [17]:
# jsondf = spark.read.json(sc.parallelize([newJson]))
jsondf = spark.read.json(spark.sparkContext.parallelize([jsonTweet]))
jsondf.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
## Write pandasUDF to to through each value in df and extract the coordinates

# After watching the VIDEO

In [ ]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("startingOffsets", "latest") \
    .option("subscribe", "twitter_status_connect") \
    .load()


df.printSchema()

topicSchema = StructType() \
                .add("schema", StringType()) \
                .add("payload", StringType())


tweets = df.select(col("key").cast("string"),
                    from_json(col("value").cast("string"), topicSchema).alias("data"), 
                    col("timestamp"))\
                    .writeStream\
                    .format("memory")\
                    .queryName("tweets_data")\
                    .outputMode("update")\
                    .start()
print(type(tweets))

In [ ]:
type(df)

In [ ]:
type(tweets)

In [ ]:
tweets.isActive

In [ ]:
df = spark.sql(""" SELECT * FROM tweets_data """)

In [ ]:
df.select('dataL', 'timestamp').show(truncate=False)

In [ ]:
df.count()

In [ ]:
df.show()

In [ ]:
df = df.orderBy("timestamp", ascending=False)

In [ ]:
df = df.limit(1)
df.show()

In [ ]:
# spark.catalog.clearCache()

In [ ]:
df.toPandas()['data']

In [ ]:
df.toPandas()['data'][0].asDict()["payload"]

In [ ]:
payload = json.loads(df.toPandas()["data"][0].asDict()["payload"])

In [ ]:
payload.get('Text')

In [ ]:
geo = payload.get('GeoLocation', None)
print(geo)

In [ ]:
df.count()

In [ ]:
type(df.toPandas()["data"])
datadf = df.toPandas()["data"].to_frame()
df.toPandas()["data"][0].asDict()["payload"]

In [ ]:
for i in range(df.count()):
    try:
        payload = json.loads(df.toPandas()["data"][i].asDict()["payload"])
        print(payload)
        if payload["GeoLocation"] != None:
            print(payload["GeoLocation"])
    except Exception as e:
        print(repr(e))
        continue

In [ ]:
tweets.stop()